In [ ]:
!nvidia-smi

Sat Oct 14 04:06:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-w13jalmb
  Running command git clone --filter=blob:none --quiet https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-w13jalmb
  Resolved https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to commit 720a87a1ee557d8ed8d9a021adbdd1dd5616c5f9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 10

In [ ]:
from parrot import Parrot
import torch
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

'''
uncomment to get reproducable paraphrase generations
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)
'''

#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# phrases = ["Loan applications can be rejected by banks for various reasons, including a low credit score, insufficient income, existing loans, and discrepancies in the provided documents. It's essential to maintain a good credit history and meet eligibility criteria for loan approval."]

# for phrase in phrases:
#   print("-"*100)
#   print("Input_phrase: ", phrase)
#   print("-"*100)
#   try:
#       para_phrases = parrot.augment(input_phrase=phrase,
#                                use_gpu=True,
#                                do_diverse=False,             # Enable this to get more diverse paraphrases
#                                adequacy_threshold = 0.8,   # Lower this numbers if no paraphrases returned
#                                fluency_threshold = 0.8)
#       for para_phrase in para_phrases:
#           print(para_phrase[0])
#           #print(type(para_phrase))
#   except:
#       print("No paraphrases returned")

In [ ]:
def sentence_permutator(sentence:str) -> list:

  sents = sentence.split('. ', 1)

  if len(sents) > 1:

    concat = []

    for sent in sents:
      #print(sent)
      tmp = []

      try:
        para_phrases = parrot.augment(input_phrase=sent,
                                use_gpu=True,
                                do_diverse=False,             # Enable this to get more diverse paraphrases
                                adequacy_threshold = 0.8,     # Lower this numbers if no paraphrases returned
                                fluency_threshold = 0.8)

        for para_phrase in para_phrases:
            #print(para_phrase[0])
            tmp.append(para_phrase[0])

      except Exception as e:
          print("No paraphrases returned --------------------------------->", e)

      # Remove duplicates and append
      concat.append(list(set(tmp)))

    # for value in concat:
    #   print(value)
    # print(len(concat[0]), len(concat[1]))

    responses = []

    for word1 in concat[0][::]:
      for word2 in concat[1][::]:
        response = word1.capitalize() + '. ' + word2.capitalize() + '.'
        responses.append(response)

    # for resp in responses:
    #   print(resp)

    return responses

  else: #If response is in single sentence

    tmp = []

    try:
      para_phrases = parrot.augment(input_phrase=sentence,
                              use_gpu=True,
                              do_diverse=False,             # Enable this to get more diverse paraphrases
                              adequacy_threshold = 0.5,     # Lower this numbers if no paraphrases returned
                              fluency_threshold = 0.8)

      for para_phrase in para_phrases:
          #print(para_phrase[0])
          tmp.append(para_phrase[0].capitalize())

    except Exception as e:
        print("No paraphrases returned --------------------------------->", e)

    return list(set(tmp))



In [ ]:
# sentence_permutator("The National Stock Exchange (NSE) is one of India's leading stock exchanges. It plays a significant role in facilitating trading in equities, derivatives, and other financial instruments. It also sets standards and regulations for the securities market.")

In [ ]:
data = pd.read_csv('Banking.csv')
data.drop('text', axis=1, inplace=True)
data.head()

,question,answer
0,What is a bank account?,A bank account is a deposit account held at a ...
1,What are the different types of bank accounts ...,There are many different types of bank account...
2,How do I open a bank account in India?,"To open a bank account in India, you need to v..."
3,What are the benefits of having a bank account?,There are many benefits to having a bank accou...
4,What is the difference between a savings accou...,A savings account is designed for storing mone...


In [ ]:
df = data.head(0)
df['false_index'] = ''
df.head()

,question,answer,false_index


In [ ]:
for index, row in data.iterrows():

  if index > 500:

    question = row['question']
    answer = row['answer']

    if index % 500 == 0:
      df.to_csv(f'Answer_Paraphrase{index}.csv', index=False)
      print('\n ------------------------------------------------------------------- Backup Generated \n')

    responses = sentence_permutator(answer)

    for resp in responses:
      df = df.append({'question' : question, 'answer' : resp, 'false_index' : index}, ignore_index=True)

    print(index, '---> Responses', len(responses))

df.to_csv(f'Answer_Paraphrase_Final.csv', index=False)

501 ---> Responses 7
502 ---> Responses 5
503 ---> Responses 6
504 ---> Responses 1
505 ---> Responses 5
506 ---> Responses 9
507 ---> Responses 4
508 ---> Responses 1
509 ---> Responses 3
510 ---> Responses 9
511 ---> Responses 2
512 ---> Responses 4
513 ---> Responses 6
514 ---> Responses 8
515 ---> Responses 3
516 ---> Responses 1
517 ---> Responses 7
518 ---> Responses 7
519 ---> Responses 7
520 ---> Responses 6
521 ---> Responses 6
522 ---> Responses 7
523 ---> Responses 4
524 ---> Responses 1
525 ---> Responses 5
526 ---> Responses 4
527 ---> Responses 6
528 ---> Responses 1
529 ---> Responses 32
530 ---> Responses 7
531 ---> Responses 8
532 ---> Responses 8
533 ---> Responses 5
534 ---> Responses 8
535 ---> Responses 5
536 ---> Responses 6
537 ---> Responses 6
538 ---> Responses 1
539 ---> Responses 7
540 ---> Responses 8
541 ---> Responses 9
542 ---> Responses 6
543 ---> Responses 1
544 ---> Responses 7
545 ---> Responses 2
546 ---> Responses 7
547 ---> Responses 1
548 ---> Res

In [ ]:
df.head()

,question,answer,false_index
0,What is a bank account?,A bank account is a deposit account in a finan...,0
1,What is a bank account?,A bank account is a deposit account that is he...,0
2,What is a bank account?,A bank account is a deposit account held in a ...,0
3,What is a bank account?,A bank account is a bank account that holds a ...,0
4,What is a bank account?,A bank account is a bank account held at a fin...,0
